In [1]:
import sys
sys.path.append("../pseudosplit")

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt
%matplotlib qt

In [5]:
from basis import FourierBasis
from state import State
from scheme import (Strang, Neri, Yoshida6,
                    AffineSym2, AffineSym4, AffineSym6, AffineSym8) 

In [6]:
from solver import Solver

In [7]:
from models import NLSE3Model1D

In [8]:
# List of splitting methods
methods_list = [Strang(), Neri(), Yoshida6(),
                AffineSym2(), AffineSym4(), AffineSym6()]

In [9]:
t0 = 0.0       # Initial time 
tf = 100000.0  # Final time
nfev = tf / 2.5e-2 * 12  #  Desired number of function evaluations
nfev_step = np.array((2, 4, 8, 2, 6, 12))  # Number of function evaluations per step for each method
dt_list = tf * nfev_step / nfev            # Corresponding time steps

In [10]:
# Soliton parameters
eta = 1.0  # amplitude
c = 0.001  # speed
omega = 0.5*(c**2 - eta**2)  # angular frequency

# Define the soliton function
soliton = lambda x: eta / np.cosh(eta*x) * np.exp(1j*c*x)

In [11]:
# Set the pseudo-spectral basis
N = 2**11  # Number of Fourier modes
I = (-50*np.pi, 50*np.pi)  # Interval
fb = FourierBasis('fb', N, I)

In [12]:
# Define the initial state
u0 = State(name='u0', basis=fb, u=soliton) 

In [13]:
# Set the model
model = NLSE3Model1D()
H = model.get_hamiltonian(fb)   # Get the hamiltonian function in the Fourier basis

In [14]:
plt.style.use("classic")
fig_H, ax_H = plt.subplots()
fig_M, ax_M = plt.subplots()

ax_H.grid()
ax_M.grid()

In [15]:
for i, method in enumerate(methods_list):
    # Console output
    dt = dt_list[i]
    print(method.name, dt)
    
    # Create the solver
    solver = Solver(model, method)

    # Initialize solver
    solver.start(u0, t0, tf) 

    # First point of trajectory
    t_values = [t0]
    H_values = [H(u0)]
    M_values = [u0.norm()]

    count = 0
    # Simulation's main loop
    while solver.active:
        u = solver.step(dt)
        t = solver.sim_time
        t_values.append(t)
        H_values.append(H(u))
        M_values.append(u.norm()) 
        count += 1
        if count % 10000 == 0:
            print(t)
            count = 0
    print(method.P_A.nfev)
    
    # Compute the relative errors for mass and hamiltonian
    H_rel_err = np.abs((H_values - H_values[0]) / H_values[0])
    M_rel_err = np.abs((M_values - M_values[0]) / M_values[0])

    # Plots
    label = "$\mathtt{" + method.name + "}, \Delta t=" + str(dt) + "$"
    ax_H.plot(t_values, H_rel_err, label=label)
    ax_M.plot(t_values, M_rel_err, label=label)
    fig_H.show()
    fig_M.show()
        
        

strang 0.004166666666666667
41.66666666667286
83.33333333333971
124.99999999996848
166.66666666659725
208.33333333322602
249.9999999998548
291.6666666664836
333.33333333311236
374.99999999974114
416.6666666663699
458.3333333329987
499.99999999962745
541.666666666661
583.3333333338581
625.0000000010554
666.6666666682526
708.3333333354498
750.000000002647
791.6666666698442
833.3333333370414
875.0000000042386
916.6666666714358
958.333333338633
1000.0000000058302
1041.6666666725453
1083.3333333386056
1125.000000004666
1166.6666666707263
1208.3333333367866
1250.000000002847
1291.6666666689073
1333.3333333349676
1375.000000001028
1416.6666666670883
1458.3333333331486
1499.999999999209
1541.6666666652693
1583.3333333313296
1624.99999999739
1666.6666666634503
1708.3333333295107
1749.999999995571
1791.6666666616313
1833.3333333276917
1874.999999993752
1916.6666666598123
1958.3333333258727
1999.999999991933
2041.6666666579933
2083.333333324054
2124.9999999901142
2166.6666666561746
2208.333333322

In [30]:
ax_H.legend(loc="upper left")
ax_M.legend(loc="upper left")

qt.qpa.xcb: QXcbConnection: XCB error: 3 (BadWindow), sequence: 36625, resource id: 11334388, major code: 40 (TranslateCoords), minor code: 0


In [19]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [20]:
plt.plot(u.grid, np.abs(u.values))

In [21]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [22]:
from scipy.fft import fftshift

In [23]:
plt.plot(fftshift(np.abs(u0.coeffs)))

In [24]:
ax_H.set_ylabel("$\epsilon_H$", fontsize=18)
ax_M.set_ylabel("$\epsilon_M$", fontsize=18)
ax_H.set_xlabel("$t$", fontsize=18)
ax_M.set_xlabel("$t$", fontsize=18)

Text(0.5, 0, '$t$')